In [1]:
import numpy as np
import csv, os
from torch.utils.data import DataLoader
from dgl.data.utils import split_dataset
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy import stats

from src_chung.model import reactionMPNN, training, inference
from src_chung.dataset import GraphDataset
from src_chung.util import collate_reaction_graphs

/home/phuocchung123/anaconda3/envs/GIN_korea/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = GraphDataset('./data_chung/data_valid2.npz')
valid_loader = DataLoader(
        dataset=data,
        batch_size=32,
        shuffle=True,
        collate_fn=collate_reaction_graphs,
        drop_last=True,
    )

In [3]:
node_dim = valid_loader.dataset.rmol_node_attr[0].shape[1]
edge_dim = valid_loader.dataset.rmol_edge_attr[0].shape[1]

In [4]:
for batch in valid_loader:
    # if batch[-1].size()[0] != 32:
    a=[b for b in batch[:23]]
    b=[b for b in batch[23:23+6]]
    break

/mnt/d/workspace/kaggle_recat/src_chung/util.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  labels = torch.stack([torch.argmax(y) for y in torch.Tensor(batchdata[-1])], axis=0)


In [32]:
from src_chung.model import reactionMPNN
import torch
de=reactionMPNN(node_dim,edge_dim)
x=torch.cat([de.mpnn(i) for i in a])
# for mol in a:
#     x.append(de.mpnn(mol))
# len(x)

torch.Size([288, 300])
torch.Size([375, 300])
torch.Size([189, 300])
torch.Size([174, 300])
torch.Size([201, 300])
torch.Size([53, 300])
torch.Size([37, 300])
torch.Size([184, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])


In [26]:
x.shape

torch.Size([1981, 300])

In [18]:
import torch

# Suppose you have the following list of tensors
list_of_tensors = [torch.tensor([1, 2, 3]), torch.tensor([4, 5, 6]), torch.tensor([7, 8, 9])]

# You can create a new tensor using torch.stack()
new_tensor = torch.stack(list_of_tensors)

print(new_tensor)


tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


In [23]:
import torch

# Your tensors
tensor1 = torch.tensor([[1, 2, 3],[4,5,6],[20,21,22]])
tensor2 = torch.tensor([[10,11,12],[30,31,32]])
tensor3 = torch.tensor([7,8,9])
print(tensor3.shape)
# Expand dimensions of tensor2 and tensor3
tensor2 = tensor2.unsqueeze(0)
tensor3 = tensor3.unsqueeze(0)
print(tensor2.shape)

# Now you can concatenate them
result = torch.cat((tensor1, tensor2, tensor3))

print(result)


torch.Size([3])
torch.Size([1, 2, 3])


RuntimeError: Tensors must have same number of dimensions: got 2 and 3